#### packets and imports

In [55]:

import math
import random
import numpy as np
import pandas as pd
import polars as pl #experimental, maybe suspicious
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display # remove later on
import ipywidgets as widgets # remove later on when dimension of views are settled


#### **1. Daten vorbereiten**  
- **Daten herunterladen** von den Weltbank-Quellen (Militärausgaben, Import/Export, BIP).  
- **Daten zusammenführen**:  
  - Eine Tabelle erstellen mit: **Land, Jahr, Militärausgaben (% BIP), Export (% BIP), Import (% BIP), BIP (absolut)**.  
  - Fehlende Werte: Entweder löschen oder durch den Durchschnitt ersetzen (einfache Lösung).

In [56]:
# reading in the data
exports_df = pl.read_csv("./data/Exports of goods and services/API_NE.EXP.GNFS.ZS_DS2_en_csv_v2_2857.csv")
gdp_df = pl.read_csv("./data/GDP/API_NY.GDP.MKTP.CD_DS2_en_csv_v2_2752.csv")
imports_df = pl.read_csv("./data/Imports of goods and services/API_NE.IMP.GNFS.ZS_DS2_en_csv_v2_2862.csv")
military_df = pl.read_csv("./data/Military expenditure/API_MS.MIL.XPND.CD_DS2_en_csv_v2_6285.csv")
meta_df = pl.read_csv("./data/Exports of goods and services/Metadata_Country_API_NE.EXP.GNFS.ZS_DS2_en_csv_v2_2857.csv")

def merge_meta(main_pl_df):
    # Convert to pandas for easier merge, then back to polars
    main_pd = main_pl_df.to_pandas()
    meta_pd = meta_df.to_pandas()

    # Merge on "Country Code"
    merged = main_pd.merge(meta_pd, left_on="Country Code", right_on="Country Code", how="left")

    # Convert back to polars DataFrame
    main_pl_df = pl.from_pandas(merged)

    return (main_pl_df)

exports_df = merge_meta(exports_df)
gdp_df = merge_meta(gdp_df)
imports_df = merge_meta(imports_df)
military_df = merge_meta(military_df)

In [57]:
# remove columns of no significance

def select_cols_with_variation(df):
    cols_with_variation = [col for col in df.columns if df.select(pl.col(col)).n_unique() > 1]
    return df.select(cols_with_variation)

exports_df = select_cols_with_variation(exports_df)
gdp_df = select_cols_with_variation(gdp_df)
imports_df = select_cols_with_variation(imports_df)
military_df = select_cols_with_variation(military_df)

In [58]:
# find Rows that have no datapoints (all year columns are empty)
def get_no_data_rows(df):
    year_cols = [col for col in df.columns if col.isdigit()]
    return df.filter(
        pl.all_horizontal([
            pl.col(col).is_null() | (pl.col(col).cast(pl.Utf8).str.strip_chars().eq(""))
            for col in year_cols
        ])
    )

no_data_exports = get_no_data_rows(exports_df)
no_data_gdp = get_no_data_rows(gdp_df)
no_data_imports = get_no_data_rows(imports_df)
no_data_military = get_no_data_rows(military_df)

# Filter out rows with no data in any of the year columns
# Union of all country codes with no data in every dataframe
no_data_codes_individually = set(no_data_exports["Country Code"].to_list()) | \
                             set(no_data_gdp["Country Code"].to_list()) | \
                             set(no_data_imports["Country Code"].to_list()) | \
                             set(no_data_military["Country Code"].to_list())

# TODO which of these countries are actually relevant for the analysis?

# Filter all dataframes to exclude these rows
# Intersection of all country codes with no data in every dataframe
no_data_codes_anywhere = set(no_data_exports["Country Code"].to_list()) & \
                         set(no_data_gdp["Country Code"].to_list()) & \
                         set(no_data_imports["Country Code"].to_list()) & \
                         set(no_data_military["Country Code"].to_list())

# Filter all dataframes to exclude these rows
exports_df = exports_df.filter(~pl.col("Country Code").is_in(no_data_codes_individually))
gdp_df = gdp_df.filter(~pl.col("Country Code").is_in(no_data_codes_individually))
imports_df = imports_df.filter(~pl.col("Country Code").is_in(no_data_codes_individually))
military_df = military_df.filter(~pl.col("Country Code").is_in(no_data_codes_individually))


In [59]:

year = widgets.IntSlider(
    value=2023,
    min=1960,
    max=2023,
    step=1,
    description='Displayed Year:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
display(year)

entries = widgets.IntSlider(
    value=20,
    min=1,
    max=gdp_df.height,
    step=1,
    description='number of entries displayed:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
display(entries)

groups = widgets.SelectMultiple(
    options=['World', 'Grouped States', 'incomes', 'Countries'],
    value=['Countries'],
    #rows=10,
    description='kinds of entries, use ctrl+click to select multiple:',
    disabled=False
)
display(groups)

output = widgets.Output()
display(output)

def balken_gdp(selected_year):
    # Only keep rows with valid GDP data for the selected year
    col_vals = gdp_df.filter(
        ~pl.col(selected_year).is_null() & (pl.col(selected_year).str.strip_chars() != "")
    ).select(["Country Name", "Country Code", selected_year, "TableName", "SpecialNotes"]).to_pandas()

    # Convert to numeric, coerce errors to NaN, drop NaN
    col_vals[selected_year] = pd.to_numeric(col_vals[selected_year], errors='coerce')
    col_vals = col_vals.dropna(subset=[selected_year])
    
    inc_vals = col_vals[
        col_vals["TableName"].str.contains("income", case=False, na=False) &
        ~col_vals["TableName"].str.contains("excluding", case=False, na=False) |
        col_vals["TableName"].str.contains("dividend", case=False, na=False) |
        col_vals["TableName"].str.contains("income", case=False, na=False)
    ] 

    world = col_vals[col_vals["Country Code"] == "WLD"]

    org_vals = col_vals[
        (col_vals["SpecialNotes"].str.contains("aggregate", case=False, na=False) |
         col_vals["TableName"].str.contains("member", case=False, na=False)) &
        ~col_vals["TableName"].str.contains("income", case=False, na=False) &
        ~col_vals.index.isin(world.index)
    ]

    country_vals = col_vals[
        ~col_vals.index.isin(inc_vals.index) &
        ~col_vals.index.isin(org_vals.index) &
        ~col_vals.index.isin(world.index)
    ]

    selected_groups = set(groups.value)

    dfs = []
    if "World" in selected_groups:
        dfs.append(world)
    if "Grouped States" in selected_groups:
        dfs.append(org_vals)
    if "incomes" in selected_groups:
        dfs.append(inc_vals)
    if "Countries" in selected_groups:
        dfs.append(country_vals)
    if dfs:
        col_vals = pd.concat(dfs, ignore_index=True)
    else:
        col_vals = pd.DataFrame(columns=col_vals.columns)

    entries.max = col_vals.shape[0]

    top_countries = col_vals.sort_values(selected_year, ascending=False)
    if top_countries.shape[0] > entries.value:
        top_countries = top_countries.head(entries.value)
    plt.figure(figsize=(10, entries.value / 4))
    sns.barplot(data=top_countries, x=selected_year, y="Country Name", orient="h")
    plt.title(f"GDP by Country in {selected_year}")
    plt.xlabel("GDP (current US$)")
    plt.ylabel("Country")
    plt.tight_layout()
    plt.show()

def on_year_change(change):
    with output:
        output.clear_output(wait=True)
        selected_year = str(change['new'])
        balken_gdp(selected_year)

def on_groups_change(change):
    with output:
        output.clear_output(wait=True)
        balken_gdp(str(year.value))

def on_entries_change(change):
    with output:
        output.clear_output(wait=True)
        balken_gdp(str(year.value))



year.observe(on_year_change, names='value')
groups.observe(on_groups_change, names='value')
entries.observe(on_groups_change, names='value')

# Initial call to display the first plot
on_year_change({'new': year.value})


IntSlider(value=2023, description='Displayed Year:', max=2023, min=1960)

IntSlider(value=20, description='number of entries displayed:', max=204, min=1)

SelectMultiple(description='kinds of entries, use ctrl+click to select multiple:', index=(3,), options=('World…

Output()

#### **2. Erste Einblicke (Deskriptive Statistik)**  
- **Mittelwerte & Streuung** berechnen:  
  - Wie hoch sind durchschnittliche Militärausgaben, Exporte und Importe?  
- **Einfache Grafiken** erstellen:  
  - **Scatterplot**: Militärausgaben (x-Achse) vs. Export/Import (y-Achse).  
  - **Boxplot**: Zeigt, ob Länder mit hohen Militärausgaben auch mehr handeln.  

#### **3. Korrelation berechnen**  
- **Pearson-Korrelation** (linearer Zusammenhang):  
  - Zwischen Militärausgaben (% BIP) und Export/Import (% BIP).  
  - Interpretation:  
    - **0 bis 0.3**: Schwacher Zusammenhang.  
    - **0.3 bis 0.7**: Mittlerer Zusammenhang.  
    - **> 0.7**: Starker Zusammenhang.  
  - *Beispielcode in R*: `cor.test(data$Militär, data$Export)`  

#### **4. Einfache Regression (falls Korrelation vorhanden)**  
- **Lineare Regression**:  
  - Export (oder Import) = Konstante + β · Militärausgaben.  
  - Fragestellung: **Erklären Militärausgaben einen Teil des Handels?**  
  - *Beispielcode in R*: `lm(Export ~ Militär, data = data)` → Prüfe **p-Wert** (Signifikanz) und **R²** (Erklärungsgrad).  

#### **5. Ausreißer checken**  
- **Offensichtliche Extremwerte** identifizieren (z. B. USA, China, kleine Ölstaaten).  
- **Sensitivitätsanalyse**: Regression einmal **mit** und einmal **ohne** Ausreißer rechnen – ändert sich das Ergebnis?  

#### **6. Ergebnisinterpretation**  
- **Fazit ziehen**:  
  - Gibt es einen Zusammenhang? Wenn ja, positiv oder negativ?  
  - Ist er stark/schwach?  
  - **Einschränkungen**: Keine Kausalität, nur Korrelation!  


### **Tools & Zeitplan**  
- **Excel/R/Python** (je nach Kenntnisstand – Excel reicht für Korrelationen).  
- **1 Woche** Daten sammeln & aufbereiten.  
- **1 Woche** Analysen & Grafiken.  
- **1 Woche** Ergebnisse zusammenfassen.  

### **Warum das ausreicht?**  
- Bachelorarbeit muss keine Nobelpreis-Methodik haben – **einfach, sauber, nachvollziehbar** ist key.  
- Korrelation + Regression sind Standardmethoden, die jeder Gutachter versteht.  

Falls ihr mehr Zeit habt, könnt ihr noch **Ländervergleiche** (z. B. "Demokratien vs. Autokratien") oder **Zeittrends** einbauen – aber das obige reicht für eine solide Basis!